# Winter Training Script

This is a training script for the evaluation function of the Winter chess engine.
It is only slightly modified compared to what I have been using in internal development.

The script relies on Numpy, Pandas, Tensorflow 2.0 and tensorflow_probability. You should use a Python 3 kernel.

I will try to keep this script roughly up to date with the 

## Imports

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_probability as tfp

In [ ]:
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras import Model

## Data Loading and Preprocessing

In the following cells we load and preprocess our training data.

Training data should be stored with Pandas as an array where the first two collumns consist of the win and the win+draw collumns and the remaining collumns consist of the position features.

In order generate such matrices, you can use pgn-extract to transform PGNs into a format that Winter can understand. Then you may feed Winter the generated files which will create a .csv file. Finally you can use pandas to transform the .csv to a more compact format to use with this script.

The pipeline to generate and integrate a new neural net in Winter is roughly as follows:

1. Get and compile the latest [pgn-extract](https://www.cs.kent.ac.uk/people/staff/djb/pgn-extract/) by David J. Barnes.
2. Use pgn-extract on your .pgn file with the arguments `-Wuci` and `--notags`. This will create a file readable by Winter.
3. Run Winter from command line. Call `gen_eval_csv filename out_filename` where filename is the name of the file generated in 2. and out_filename is what Winter should call the generated file. This will create a .csv dataset file (described below) based on pseudo-quiescent positions from the input games.
4. Either transform the .csv to a pandas format or modify this script to read out the information in the format you prefer.
5. Run this training script
6. Copy the output of the last cell and use it as a replacement for the contents of Winter's src/net_weights.h file
7. Recompile Winter with the new weights.

Depending on what you are doing (eg. modifying the architecture) you may need to do some more work than that.

In [ ]:
def np_to_tensor(val):
  val_np = np.asarray(val, np.int8)
  return tf.convert_to_tensor(val_np, tf.int8)

**Modify the next cell to match your input data.**

In [ ]:
X = np_to_tensor(pd.read_pickle('./nn_inputsetv26.bz2', compression='bz2'))

If you did everything correctly, the first two collumns should be the win and win+draw probabilities respectively. These in turn imply W/D/L probabilities and are currently the output of the model that Winter expects.

In [ ]:
Y = X[:,0:2]
Y = tf.cast(Y, tf.float32)
X = X[:,2:]

The scale parameter is used to rescale input features in the desired way. This is integrated into the weights at the end in order to save computational overhead and code complexity.

While the function used here is likely not optimal, I have played around a bit with it and not found anything better.

In [ ]:
scale = 1 / tf.math.maximum(tf.cast(tf.reduce_max(X, axis=0), tf.float32), 1)

If you are using the default scale function, then the inverse gives us and understanding of the expected range of values for each feature.

In [ ]:
1 / scale

If you set things up identically to how I do, then the very first sample is the standard starting position. I think this is a simple base case to check to make sure features make sense.

In [ ]:
X[0,:]

## Train Test Split

In [ ]:
num_test = 20000

X_train = X[num_test:,:]
X_test = X[:num_test,:]

Y_train = Y[num_test:]
#Y_train = Y_train[..., tf.newaxis]
Y_test = Y[:num_test]
#Y_test = Y_test[..., tf.newaxis]

In [ ]:
Y_train.shape

In [ ]:
train_ds = tf.data.Dataset.from_tensor_slices(
    (X_train, Y_train)).shuffle(10000).batch(128)
test_ds = tf.data.Dataset.from_tensor_slices((X_test, Y_test)).batch(32)

## Model Definition and Notes on Model Choice

The next two cells are used to define the model.

#### Some Things to Keep in Mind

Winter does not itself rely on Tensorflow or any other ML library. This means that any feature that is reflected in the final model must get supported in the Winter codebase. Should you need support for some feature, please raise an issue on github or contact me personally. Unfortunately I cannot guarantee support for arbitrary functions.

Winter is designed for play on CPU. Attempting to train large models with batching on GPU in mind is not wise for this reason as I do not have access to a GPU at all for this project. I would be willing to help support a GPU branch of Winter, but for this reason I cannot be the primary contributor to such a feature.

The last cell in this script generates a source file that can be integrated into Winter. Longterm I would like to support ONNX and loading external models, but for now I would like to stick to small models that can be backed into Winter's source directly. Modifying the model definition in the following cells may also require changing the last cell.

In [ ]:
class SpectralNormalization(tf.keras.layers.Wrapper):
    """
    Attributes:
       layer: tensorflow keras layers (with kernel attribute)
    """

    def __init__(self, layer, **kwargs):
        super(SpectralNormalization, self).__init__(layer, **kwargs)

    def build(self, input_shape):
        """Build `Layer`"""

        if not self.layer.built:
            self.layer.build(input_shape)

            if not hasattr(self.layer, 'kernel'):
                raise ValueError(
                    '`SpectralNormalization` must wrap a layer that'
                    ' contains a `kernel` for weights')

            self.w = self.layer.kernel
            self.w_shape = self.w.shape.as_list()
            self.u = self.add_variable(
                shape=tuple([1, self.w_shape[-1]]),
                initializer=tf.keras.initializers.TruncatedNormal(stddev=0.02),
                name='sn_u',
                trainable=False,
                dtype=tf.float32)

        super(SpectralNormalization, self).build()

    @tf.function
    def call(self, inputs):
        """Call `Layer`"""
        # Recompute weights for each forward pass
        self._compute_weights()
        output = self.layer(inputs)
        return output

    def _compute_weights(self):
        """Generate normalized weights.
        This method will update the value of self.layer.kernel with the
        normalized value, so that the layer is ready for call().
        """
        w_reshaped = tf.reshape(self.w, [-1, self.w_shape[-1]])
        eps = 1e-12
        _u = tf.identity(self.u)
        _v = tf.matmul(_u, tf.transpose(w_reshaped))
        _v = _v / tf.maximum(tf.reduce_sum(_v**2)**0.5, eps)
        _u = tf.matmul(_v, w_reshaped)
        _u = _u / tf.maximum(tf.reduce_sum(_u**2)**0.5, eps)

        self.u.assign(_u)
        sigma = tf.matmul(tf.matmul(_v, w_reshaped), tf.transpose(_u))

        self.layer.kernel = self.w / sigma

    def compute_output_shape(self, input_shape):
        return tf.TensorShape(
            self.layer.compute_output_shape(input_shape).as_list())
SN = SpectralNormalization

In [ ]:
reg = tf.keras.regularizers.l2

class CReLU(tf.keras.layers.Layer):
  def __init__(self):
    super(CReLU,self).__init__()
    self.relu1 = tf.keras.layers.ReLU()
    self.relu2 = tf.keras.layers.ReLU()
    self.concat = tf.keras.layers.Concatenate()

  def call(self, x):
    return self.concat([self.relu1(x), self.relu2(-x)])

class MyModel(Model):
  def __init__(self, n=32):
    super(MyModel, self).__init__()
    self.d1 = Dense(n, name="d1")
    self.a1 = tf.keras.layers.ReLU()
    self.d2 = Dense(n, name="d2")
    self.a2 = tf.keras.layers.ReLU()
    self.out = Dense(2, activation='sigmoid', bias_initializer = tf.initializers.constant(value=0.0), name="out")

  def call(self, x, training=False):
    x = tf.cast(x, tf.float32) * scale
    x = self.d1(x)
    x = self.a1(x)
    x = self.d2(x)
    x = self.a2(x)
    return self.out(x)

model = MyModel(16)

## Optimization, Loss and Metrics Definition and Initialization

In [ ]:
clo = tf.keras.losses.BinaryCrossentropy()
clo2 = tf.keras.losses.MeanSquaredError()

def custom_loss(pred_y, y):
  return clo(pred_y, y) + clo2(tf.reduce_mean(pred_y, 1), tf.reduce_mean(y, 1))

In [ ]:
loss_object = tf.keras.losses.BinaryCrossentropy()

optimizer = tf.keras.optimizers.Adam(amsgrad=True)

In [ ]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.MeanAbsoluteError(name='train_accuracy')
mean_prediction = tf.keras.metrics.Mean(name='mean_prediction')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.MeanAbsoluteError(name='test_accuracy')

In [ ]:
min_train_loss = tf.math.reduce_mean(loss_object(Y_train, Y_train))
min_test_loss = tf.math.reduce_mean(loss_object(Y_test, Y_test))

In [ ]:
min_train_loss

In [ ]:
model.losses

## Training Loop Definition

In [ ]:
@tf.function
def reset_metrics():
  train_loss.reset_states()
  train_accuracy.reset_states()
  mean_prediction.reset_states()

  test_loss.reset_states()
  test_accuracy.reset_states()

In [ ]:
@tf.function
def train_step(images, labels):
  with tf.GradientTape() as tape:
    predictions = model(images, training=True)
    #regularization_loss = tf.math.add_n(model.losses)
    pred_loss = loss_object(labels, predictions)
    loss = pred_loss #+ regularization_loss
    # loss = loss_object(labels, predictions)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))
  
  mean_prediction(predictions)
  train_loss(pred_loss)
  train_accuracy(tf.reduce_mean(labels, 1), tf.reduce_mean(predictions, 1))

In [ ]:
@tf.function
def test_step(images, labels):
  predictions = model(images)
  t_loss = loss_object(labels, predictions)

  test_loss(t_loss)
  test_accuracy(tf.reduce_mean(labels, 1), tf.reduce_mean(predictions, 1))

## Initial Test Statistics

In [ ]:
for test_images, test_labels in test_ds:
  test_step(test_images, test_labels)

template = 'Initial test values are Test Loss: {}, Test Accuracy: {}'
print (template.format(test_loss.result()-min_test_loss, test_accuracy.result()*100))

## Training

In [ ]:
EPOCHS = 180

for epoch in range(EPOCHS):
  reset_metrics()
  for images, labels in train_ds:
    train_step(images, labels)

  for test_images, test_labels in test_ds:
    test_step(test_images, test_labels)

  template = 'Epoch {0:}, Loss: {1:.5f}, Test Loss: {4:.5f}, Abs Loss: {2:.5f}, Test Abs Loss: {5:.5f}, Mean pred: {3:.5f}'
  print (template.format(epoch+1,
                         train_loss.result()-min_train_loss,
                         train_accuracy.result(),
                         mean_prediction.result(),
                         test_loss.result()-min_test_loss,
                         test_accuracy.result()))

## Model Summary

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

import os

In [ ]:
model.summary()

## C++ Source File Generation

If you modify the model architecture then you will likely need to make changes here.

This should generate the source you can use to replace the net_weights.h file in Winter's source directory.

In [ ]:
print("/*")
print(" * net_weights.h")
print(" *")
print(" *  Created on: Jul 9, 2019")
print(" *      Author: Jonathan")
print(" */")
print("")
print("#ifndef SRC_NET_WEIGHTS_H_")
print("#define SRC_NET_WEIGHTS_H_")
print("")
print("#include <array>")
print("namespace net_hardcode {")
print("")
      
weights = model.out.trainable_weights

print("constexpr float bias_win = {};".format(weights[1][0]))
print("constexpr float bias_win_draw = {};".format(weights[1][1]))
print("")

print("constexpr std::array<float, {}> output_weights = {}".format(weights[0].numpy().shape[0]
                                                                    * weights[0].numpy().shape[1], "{"))
for i in range(weights[0].numpy().shape[1]):
  for j in range(weights[0].numpy().shape[0] // 4):
    k = j*4
    print("  {}, {}, {}, {},".format(weights[0][k][i], weights[0][k+1][i],
                                     weights[0][k+2][i], weights[0][k+3][i]))
print("};")
print("")

weights = model.d2.trainable_weights

print("constexpr std::array<float, {}> l2_bias = {}".format(weights[1].numpy().shape[0], "{"))
for j in range(weights[1].numpy().shape[0] // 4):
  k = j*4
  print("  {}, {}, {}, {},".format(weights[1][k], weights[1][k+1],
                                   weights[1][k+2], weights[1][k+3]))
print("};")
print("")

print("constexpr std::array<float, {}> l2_weights = {}".format(weights[0].numpy().shape[0]
                                                               * weights[0].numpy().shape[1], "{"))
for i in range(weights[0].numpy().shape[0]):
  for j in range(weights[0].numpy().shape[1] // 4):
    k = j*4
    print("  {}, {}, {}, {},".format(weights[0][i][k], weights[0][i][k+1],
                                     weights[0][i][k+2], weights[0][i][k+3]))
print("};")
print("")

weights = model.d1.trainable_weights

print("constexpr std::array<float, {}> l1_bias = {}".format(weights[1].numpy().shape[0], "{"))
for j in range(weights[1].numpy().shape[0] // 4):
  k = j*4
  print("  {}, {}, {}, {},".format(weights[1][k], weights[1][k+1],
                                   weights[1][k+2], weights[1][k+3]))
print("};")
print("")

print("constexpr std::array<float, {}> l1_weights = {}".format(weights[0].numpy().shape[0]
                                                               * weights[0].numpy().shape[1], "{"))
for i in range(weights[0].numpy().shape[0]):
  for j in range(weights[0].numpy().shape[1] // 4):
    k = j*4
    print("  {}, {}, {}, {},".format(weights[0][i][k] * scale[i], weights[0][i][k+1] * scale[i],
                                     weights[0][i][k+2] * scale[i], weights[0][i][k+3] * scale[i]))
print("};")
print("")
print("}")
print("")
print("#endif /* SRC_NET_WEIGHTS_H_ */")